# Project Timeline
    - Monday: Data acquiration 
    - Tuesday: Pineline, EDA and Visualization
    - Wednesday: Hypotheses testing and get things organized
    - Thursday: Preparation for presentation
    - Friday: Update latest data and make minor revise

-----------------


# Monday: Data acquiration
    - [x] Getting CSSE data on github
    - [x] Gather state testing data thru API
    - [x]  Webscrap korean testing data
    - [x]  Webscrap LA community level data
    - []  consolidate as database backup into psql

**Data Source:**

- **Getting CSSE data on github**

    source: https://github.com/CSSEGISandData/COVID-19 by Johns Hopkins CSSE


- **Gather state testing data thru API**

    source: https://covidtracking.com/data by Johns Hopkins CSSE
    - US Testing in Time Series https://covidtracking.com/api/us/daily.csv
    - States Historical Data https://covidtracking.com/api/states/daily.csv


- **Webscrap korean testing data**

    Source: CDC of South Korean
    - sample data https://www.cdc.go.kr/board/board.es?mid=&bid=0030&act=view&list_no=366735
    
    
- **Webscrap LA community level data**

    Source: The Department of Public Health of Los Angeles County
    - sample data http://publichealth.lacounty.gov/phcommon/public/media/mediapubhpdetail.cfm?prid=2298

**Web scraping preparion**

Firstly, create a webscrap class including functions for getting tables, get text and get urls. Such class definition is stored as ./src/web_scraping

In [2]:
from src.web_scaping import webscrap
import csv

**Gather data with webscape class**

Many experts treated South Korea as a best practice in controlling COVID-19 and an good model to follow. Hence, in this study we will compare COVID-19 data between US and South Korea.

Unfortunately, there isn't an existing data source for South Korea testing data for the coronavirus. We utilized web scaping technique to collect data from press release of each day.


In [ ]:
# Gather all urls for all korean press releases

url_p1 = "https://www.cdc.go.kr/board.es?mid=&bid=0030&nPage="
url_p2 = (str(i) for i in list(range(1,25)))

url_lst = []

for page in url_p2:
    nws = webscrap(url_p1 + page)
    nws.start(random_headers = True)
    url_lst.append(nws.get_urls())

with open("url_k_t.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(url_lst2)
    

In [ ]:
# Gather Table from each page

url_p1 = "https://www.cdc.go.kr"
out_lst = []

for page in urls2:
    nws = webscrap(url_p1 + page)
    nws.start(random_headers = True)
    out_lst.append(nws.get_table())

with open("South_Korea_History data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(out_lst)
    

Not only the big picture in a country scale that we care, we also wants to provide a LA picture community level data. Similarly, we webscaped community level data from press release from Department of Public Health of Los Angeles County.

In [ ]:
# collect LA data

url = "http://publichealth.lacounty.gov/phcommon/public/media/mediapubhpdetail.cfm?prid="
url_lst = []
for i in list(range(2300,2267,-1)):
    url_lst.append(url + str(i))
out = []
for url in url_lst:
    nws = webscrap(url)
    nws.start(random_headers = True)
    temp_lst = []
    text = nws.get_text(elements = ["ul", "li"])
    if text:
        temp_lst.append(url[-4:])
        temp_lst.append(nws.get_table(index=1)[0][0])
        temp_lst.append(text)
        out.append(temp_lst)

# Save as csv

with open("LA_backup.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(out)


### Data Cleaning

Create a data pipeline to structure data
- LA data

In [ ]:
def lacounty_daily_text_2_table(text_list):
    # pipeline function for date from mar 30
    out_lst = []
    temp = list(filter(lambda x: "City" in x, text_list))
    if temp:
        string = temp[0]
        str_lst2 = string.split("\t  ")
        string2 = str_lst2[0][:str_lst2[0].find("\t(\t")]

        for i in str_lst2:
            string2 = i[:i.find("\t(\t")]
            out_lst.append(string2.replace("\t",",").split(","))
    return out_lst

In [ ]:
def lacounty_daily_text_2_table2(text_list):
    # pipeline function for date of mar 22 - mar 29
    out_lst = []
    temp = list(filter(lambda x: "City" in x, text_list))
    if temp:
        string = temp[0]
        str_lst2 = string.split("\t")
        for i in range(len(str_lst2) - 1):
            temp_lst = []
            temp_lst.append (str_lst2[i].split("  ", 1)[1])
            temp_lst.append (str_lst2[i + 1].split("  ", 1)[0])
            out_lst.append(temp_lst)
    return out_lst

In [ ]:
# data pipeline after Mar 30
out2 = []
for item in out:
    temp_lst = [item[1].split("\r\n\t")[1]]
    if len(item[2]) > 0:
        temp_lst.append(lacounty_daily_text_2_table(item[2]))
    out2.append(temp_lst)

In [ ]:
# data pipeline before mar 29
out3 = []
for item in out[11:]:
    temp_lst = [item[1].split("\r\n\t")[1]]
    if len(item[2]) > 0:
        temp_lst.append(lacounty_daily_text_2_table2(item[2]))
    out3.append(temp_lst)

In [ ]:
LA_data = {}
for i in out2[:11]:
    if len(i[1]):
        LA_data[i[0]] = i[1]
for i in out3:
    if len(i[1]):
        LA_data[i[0]] = i[1]

In [ ]:
# merge into a readable format
import pandas as pd

df = pd.DataFrame(LA_data['April 06, 2020'], columns= ["city", 'April 06, 2020'] )
df.set_index(["city"])

for key in LA_data.keys():
    df2 = pd.DataFrame(LA_data[key], columns= ["city", key])
    df2.set_index(["city"])
    df = pd.merge(df, df2, how='left', left_on="city", right_on="city")



After checking the merged dataset, some data was not properly merge since the city name format has been change since March 26, 2020. We need do minor fix on the dataset.

In [ ]:
df.drop(["April 06, 2020_y", "March 26, 2020", "March 25, 2020", "March 24, 2020", "March 23, 2020", "March 22, 2020"], axis=1, inplace=True)
df

In [ ]:
# helper function to fix city names
def aka_name(city_name):
    new_name = ""
    if city_name[:7] == "City of":
        new_name += city_name[-(len(city_name) - 8) : ]
    elif city_name[:14] == 'Unincorporated':
        new_name += city_name[-(len(city_name) - 17) : ]
    elif city_name[:14] == 'Los Angeles - ':
        new_name += city_name[-(len(city_name) - 14) : ]
    else:
        new_name += city_name
    return new_name

df["city2"] = df["city"].apply(lambda x: aka_name(x))

In [ ]:
# fix data for the first 5 day data
key_list = list(LA_data.keys())[-5 : ]
for key in key_list:
    df2 = pd.DataFrame(LA_data[key], columns= ["city2", key])
    df2["city2"] = df2["city2"].apply(lambda x : x.strip("*"))
    df2.set_index(["city2"])
    df = pd.merge(df, df2, how='left', left_on="city2", right_on="city2")
    


In [ ]:
# making a csv backup
df.to_csv("la_community_0406.csv")

Get korea data into Pandas

In [7]:
with open("./Data/Testing/South_Korea_History data.csv", "r", newline="") as f:
    reader = csv.reader(f)
    k_data = list(reader)

In [64]:
tmp_lst = list(i[2] for i in k_data)
tmp_lst2 = []
for item in tmp_lst:
    tmp_lst3 = item.replace(', ', '').replace("'", '').replace("]", '').split("\\n")[1:]
    tmp_lst2.append(tmp_lst3)
    
for i in range(len(tmp_lst2)):
    for j in range(len(tmp_lst2[i])):
        tmp_lst2[i][j] = tmp_lst2[i][j].replace("As of 0:00", '').replace("As of 9:00", '').replace("As of 16:00", '').replace("As of 09:00", '')
    if len(tmp_lst2[i][0].strip()) == 0:
        tmp_lst2[i] = tmp_lst2[i][1:]
    if  " (" in tmp_lst2[i][0]:
        tmp_lst2[i][0] = tmp_lst2[i][0][: tmp_lst2[i][0].find(" (")]        
            

# Tuesday: Pineline, EDA and Visualization
    -[ ] Data cleaning and get ready for Pandas
    -[ ] Data screening and mark outlier
    -[ ] create pineline to process data
    -[ ] EDA on confirm case, death case, daily confirm case and testing data on Korean, CA, NY, US
    -[ ] Visualization on community level

In [37]:
tmp_lst[0].replace(', ', '').replace("'", '').replace("]", '').split("\\n")

['[',
 'As of 0:00',
 '5 April (Sun)',
 '461,233',
 '10,237',
 '6,463',
 '3,591',
 '183',
 '19,571',
 '431,425']

In [68]:
df = pd.DataFrame(tmp_lst2)

In [66]:
import pandas as pd

In [70]:
df.to_csv("k_data_0406.csv")